In [1]:
!pip3 install torchmetrics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torchmetrics

from sklearn.model_selection import train_test_split

Fontconfig warning: ignoring C.UTF-8: not a valid language tag


In [3]:
data_path = '~/data/all_DEFR_comments_27062022.csv'

# , dtype={'ID':float, 'kommentar_original':str, 'label':int}
df = pd.read_csv(data_path)

/scratch/tmp.30552059.ewybitul/ipykernel_1854/1032208847.py:4: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [8]:
df.head(3)

,ArticleID,ID,geschlecht,alter,sexualitaet,religion,nationalitaet,beeintraechtigung,sozialer_status,politik,aussehen,andere,toxische_sprache,label,kommentar_original,kommentar
0,395000000000000000.0,977000000000000000.0,0,0.0,0,0,0,0.0,0,0,0,1,0,1,Ich denke sie sollten sich erhängen. Wo haben ...,ich denke sie sollten sich erhängen. wo haben ...
1,479000000000000000.0,1010000000000000000.0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,"Macjst du Trolle aus normale Menschen machen, ...","macjst du trolle aus normale menschen machen, ..."
2,490000000000000000.0,1556419213.0,0,0.0,0,1,0,0.0,0,0,0,0,0,1,"Christentum.hat weder Sharia noch Koran , die ...","christentum.hat weder sharia noch koran , die ..."


In [4]:
# Load model directly
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss")
model = AutoModelForSequenceClassification.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss", num_labels=1)

Some weights of the model checkpoint at statworx/bert-base-german-cased-finetuned-swiss were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint 

In [53]:
df_train = df[:-1000]
test = df[-1000:]
train, val = train_test_split(df_train, test_size=0.2, random_state=42, shuffle=True)

train_X, train_y = train["kommentar"], train["label"]
test_X, test_y = test["kommentar"], test["label"]
val_X, val_y = val["kommentar"], val["label"]

In [56]:
class MyBatchedDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data.iloc[index]
        y = torch.tensor(self.labels.iloc[index], dtype = torch.float32)
        return x, y

train_dataset = MyBatchedDataset(train_X[:1000], train_y[:1000])
val_dataset = MyBatchedDataset(val_X, val_y)
test_dataset = MyBatchedDataset(test_X, test_y)
batch_size = 16
val_batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=val_batch_size, shuffle=False)

In [57]:
# Batched
import math
from torchmetrics.classification import BinaryF1Score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
torch.manual_seed(0)


#tokenizer = AutoTokenizer.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss")
#model = AutoModelForSequenceClassification.from_pretrained("statworx/bert-base-german-cased-finetuned-swiss", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-german")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-german", num_labels = 2)
# tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse")
# model = AutoModelForSequenceClassification.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse", num_labels=2)

#tokenizer = AutoTokenizer.from_pretrained("german-nlp-group/electra-base-german-uncased")
#model = AutoModelForSequenceClassification.from_pretrained("german-nlp-group/electra-base-german-uncased")

#NUM_CLASSES = 2
#model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=NUM_CLASSES)

#criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=1e-5)


# Freeze specific sublayers
freeze_sublayers = ['encoder.layer.0.', 'encoder.layer.1.', 'encoder.layer.2.',
                    'encoder.layer.3.', 'encoder.layer.4.', 'encoder.layer.5.',
                    'encoder.layer.6.', 'encoder.layer.7.', 'encoder.layer.8.',
                    'encoder.layer.9.', 'encoder.layer.10.']
for name, param in model.named_parameters():
    for freeze_layer in freeze_sublayers:
        if freeze_layer in name:
            param.requires_grad = False

dif_learning_rate = [3e-4, 7e-5]
#optimizer = torch.optim.SGD([{'params': model.bert.encoder.parameters(), 'lr': dif_learning_rate[1]},
#                                             {'params': model.classifier.parameters(), 'lr': dif_learning_rate[0]}], momentum=0.9)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)


model.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

True

True

In [58]:
import tqdm
import random

In [65]:
loss = None
epochs = 3
best_val_loss = math.inf
iter_no_change = 0

model.train()

for idx in range(epochs):
    losses = 0
    
    for batch_X, batch_Y in tqdm.tqdm(train_loader): 
        if random.random() < 0.999:
            pass
        batch_X = [str(x) for x in batch_X]
        batch_X = tokenizer.batch_encode_plus(batch_X, padding=True, truncation=True, return_tensors='pt')['input_ids']
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)


        logits = outputs.logits.squeeze()
        batch_Y = torch.squeeze(batch_Y).long()
        loss = criterion(logits, batch_Y)

        loss.backward()
        optimizer.step()

        losses += loss.item()

    print(f"Train loss after {idx: 04d} / {epochs: 04d} epochs: {losses/len(train_loader): 0.4f}")

    with torch.no_grad():
        model.eval()
        val_loss = 0
        targets, outs = [], []
        for valid_X, valid_Y in val_loader:
            valid_X = [str(x) for x in batch_X]
            valid_X = tokenizer.batch_encode_plus(valid_X, padding=True, truncation=True, return_tensors='pt')['input_ids']
            valid_X, valid_Y = valid_X.to(device), valid_Y.to(device)

            outputs = model(valid_X)
            print(outputs)
            logits = outputs.logits
            # logits = torch.squeeze(logits)

            # outs.extend([logits > 0])
            # targets.extend(valid_Y)
            # valid_Y = valid_Y.unsqueeze(0)
            # valid_Y = valid_Y[0, :]
            # valid_Y = torch.squeeze(valid_Y).long()
            val_loss += criterion(logits, valid_Y).item()
            
        val_loss = val_loss / len(val_loader)
        scheduler.step(val_loss)
        print(f"  Val loss after {idx: 04d} / {epochs: 04d} epochs: {val_loss: 0.4f}")

        f1 = BinaryF1Score()
        #outs_t, targets_t = torch.cat(outs, dim=0).long(), torch.cat(targets, dim=0).long()

        #print(targets_t.shape)
        #print(outs_t.shape)

        #print(f"  Val F1 after {idx: 04d} / {epochs: 04d} epochs: {f1(outs_t, targets_t): 0.4f}")

        if val_loss < best_val_loss:
            torch.save(model, f'models/swiss_bert_model_{val_loss}.pt')
            print("model saved!")
#                     torch.save(classifier, 'ger_classifier.pt')
            best_val_loss = val_loss
            iter_no_change = 0
        if val_loss >= best_val_loss:
            iter_no_change += 1
            #if iter_no_change > 10:
                #print(f"Early stopping: val loss didn't improve for {10*100*idx} steps!")
                #break
    #if iter_no_change > 10:
        #break

 21%|██        | 13/63 [00:01<00:06,  8.09it/s]


TypeError: must be real number, not str